### Add model, connection and device to settings:

In [6]:
# библиотеки
import os
import sys
import inspect
import numpy as np
import re
import math
import matplotlib.pyplot as plt
from ipywidgets import interactive
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from hybriddomain.envs.hs.model.model_main import ModelNet as Model
from hybriddomain.settings.settings_main import Settings

# пути и названия файлов
modelFileName = 'problems/runge_kutta_linear_system'
conn_name = "connection.json"
device_conf_name = "devices.json"

#Ввод пароля для инициализации пользователя
model = Model()
model.io.loadFromFile(modelFileName)
settings = Settings(model, conn_name, device_conf_name, conn_name)

INFO:settings_main:device_conf_rpath
INFO:settings_main:devices.json


Please enter password for user imaslenikov:


 ·········


In [7]:
print("equations:")
print(model.equations)
print("params:")
print(model.params)
print("paramValues:")
print(model.paramValues)

equations:
[["U'= a*U+b*V", "V'= c*U+d*V"]]
params:
['a', 'b', 'c', 'd']
paramValues:
[{'a': -2, 'b': 1, 'c': 2, 'd': -3}, {'a': -2, 'b': 1, 'c': 6, 'd': -3}, {'a': -1, 'b': 3, 'c': -3, 'd': -1}, {'a': 0, 'b': 2, 'c': -2, 'd': 0}, {'a': 0, 'b': 4, 'c': -4, 'd': 0}]


### Run solver:

In [8]:
# запуск процесса решения уравнения
from hybriddomain.solvers.hs.remoterun.progresses.progress_notebook import ProgressNotebook
from hybriddomain.solvers.hs.postproc.results.results_main import ResultPostprocNet as ResultPostproc
progress = ProgressNotebook(100, prefix='solving')

progress_params = ProgressNotebook(len(model.paramValues), prefix='param')
# display(progress_params.progress)

progress_solving = ProgressNotebook(100, prefix='solving')
# display(progress_solving.progress)
# progress_solving.succ(0)

result_postproc = ResultPostproc(modelFileName)

from hybriddomain.solvers.hs.remoterun.remoterun import remoteProjectRun

clear_output()
test_name = settings.paths['model']['name']
bTest = widgets.Button(description="run %s " % (test_name))

# parametrisation:
def run(event):
    model = Model()
    model.io.loadFromFile(modelFileName)
    
    for params_idx in range(len(model.paramValues)):
        
        progress_params.succ(params_idx)
        progress_solving.set_prefix("solving")
        
        # changing param:
        model.defaultParamsIndex = params_idx
        model.io.saveToFile(settings.paths["hd"]["json"])
        model = Model()
        model.io.loadFromFile(modelFileName)
        
        remoteProjectRun(settings, 1, progress_solving,remove_old=False)
        
        # result_postproc.extract_out(params_idx, results_params)
        result_postproc.rename_out(params_idx)
    display(Javascript("alert('Done');"))
    
bTest.on_click(run)

display(progress_params.progress)
display(progress_solving.progress)
 
display(bTest)

IntProgress(value=0, description='param: ', max=4)

IntProgress(value=0, description='solving: ', max=99)

Button(description='run runge_kutta_linear_system ', style=ButtonStyle())

### Results:

In [9]:
model.readResults(result_format=1)
# available result files:
print("result files: ", model.results_paths)
# available plot files:
print("plot files: ", model.plots_paths)

result files:  {'u': ['problems/runge_kutta_linear_system/out/u_seq0.out', 'problems/runge_kutta_linear_system/out/u_seq1.out', 'problems/runge_kutta_linear_system/out/u_seq2.out', 'problems/runge_kutta_linear_system/out/u_seq3.out', 'problems/runge_kutta_linear_system/out/u_seq4.out']}
plot files:  {}


In [10]:
# объекте model хранится вся информация по решению задачи
# get results (for TRAC-11 result_format=1):
model.readResults(result_format=1)
model.results_paths
# available results:
print(model.results_arrays.keys())

dict_keys(['u'])


### Video:

In [11]:
# available plots:
model.plots_paths

{}

In [12]:
display(HTML(model.get_video('System',0)))
display(HTML(model.get_video('System',1)))
display(HTML(model.get_video('System',2)))
display(HTML(model.get_video('System',3)))
display(HTML(model.get_video('System',4)))

KeyError: 'System'

### U(x, t-fixed),V(x, t-fixed):

In [14]:
# отрисовывает результат
%matplotlib inline

#display(w)
time = model.results_arrays['u']['timevalues']
#print(time)

times = model.results_arrays['u']['timevalues']
#print(times)
result = model.results_arrays['u']['resvalues']
print(result[0][0][0][10])
print(len(times))
result_x = np.array([result[0][time][0][5] for time in times])
result_y = np.array([result[0][time][1][5] for time in times])

tt =np.linspace (0.0,1.0,101)
circle_x=np.array([10*math.cos(2*3.1415926*t) for t in tt])
circle_y=np.array([10*math.sin(2*3.1415926*t) for t in tt])


def f(x,t):
    result_x = np.array([result[x][time][0][t] for time in times])
    result_y = np.array([result[x][time][1][t] for time in times])
    # Сама траектория зависимость от t берется разное начало запуска
    plt.plot(result_x, result_y)
    # Круг на котором лежат начала траекторий 
    plt.plot(circle_x, circle_y)
w = interactive(f,x=(0,len(result)-1,1),t=(0,100,1))
display(w)
print(len(result))
plt.show()

8.09017
101


interactive(children=(IntSlider(value=2, description='x', max=4), IntSlider(value=50, description='t'), Output…

5
